# PPD dos dados para treinamento, validação e teste

In [1]:
##importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random
from numpy import median, mean
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

* Outlier =  1 normal
* Outlier = -1 anormal

## IMPORTANDO dados treinamento, validacao e teste

In [2]:
# atributo DURACAO

In [3]:
## importando dataset TREINAMENTO
dfc_trein = pd.read_csv('datasetaruba/pp4/aruba-unifatrib-trein.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(dfc_trein.shape)
dfc_trein.head()

(56, 18)


,dia,dia_semana,fim_semana,BedToToilet_freq,Eating_freq,MealPreparation_freq,Relax_freq,Sleeping_freq,BedToToilet_dur,Eating_dur,MealPreparation_dur,Relax_dur,Sleeping_dur,BedToToilet_dfreq,Eating_dfreq,MealPreparation_dfreq,Relax_dfreq,Sleeping_dfreq
0,1,3,0,1.0,4.000000,9.0,9.0,2.0,158.0,626.0,4000.0,14000.0,28459.0,158,156,444,1555,14229
1,2,4,0,1.0,1.000000,4.0,9.0,3.0,160.0,209.0,1500.0,14000.0,48019.0,160,209,375,1555,16006
2,3,5,1,1.0,3.000000,4.0,9.0,1.0,189.0,1795.0,1500.0,14000.0,16215.0,189,598,375,1555,16215
3,4,6,1,2.0,2.933333,9.0,17.0,5.0,348.0,3500.0,1898.0,28849.0,47460.0,174,1193,210,1697,9492
4,5,0,0,1.0,4.000000,9.0,13.0,2.0,184.0,3224.0,2118.0,25717.0,10732.0,184,806,235,1978,5366


In [4]:
## importando dataset de TESTE
df_test2035 = pd.read_csv('datasetaruba/pp4/aruba-unifatrib-test-2035.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(df_test2035.shape)
df_test2035.head()

df_test4055 = pd.read_csv('datasetaruba/pp4/aruba-unifatrib-test-4055.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(df_test4055.shape)
df_test4055.head()

df_test6075 = pd.read_csv('datasetaruba/pp4/aruba-unifatrib-test-6075.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(df_test6075.shape)
df_test6075.head()

df_test8095 = pd.read_csv('datasetaruba/pp4/aruba-unifatrib-test-8095.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(df_test8095.shape)
df_test8095.head()

df_test2095 = pd.read_csv('datasetaruba/pp4/aruba-unifatrib-test-2095.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(df_test2095.shape)
df_test2095.head()

(28, 35)
(28, 35)
(28, 35)
(28, 35)
(28, 35)


,dia,dia_semana,fim_semana,BedToToilet_freq,Eating_freq,MealPreparation_freq,Relax_freq,Sleeping_freq,var_BedToToilet_freq,var_Eating_freq,...,MealPreparation_dfreq,Relax_dfreq,Sleeping_dfreq,var_BedToToilet_dfreq,var_Eating_dfreq,var_MealPreparation_dfreq,var_Relax_dfreq,var_Sleeping_dfreq,outlier,variacao
0,1,0,0,5.0,2.0,12.0,6.0,2.0,-1,-1,...,68,9128,44375,-1,-1,-1,-1,-1,-1,60-75
1,2,2,0,5.0,5.0,17.0,31.0,4.0,-1,-1,...,65,196,13486,-1,-1,-1,-1,-1,-1,60-75
2,3,0,0,4.0,8.0,10.0,8.0,3.0,-1,-1,...,69,5372,30929,-1,-1,-1,-1,-1,-1,60-75
3,4,3,0,5.0,5.0,8.0,6.0,4.0,-1,-1,...,98,6387,12885,-1,-1,-1,-1,-1,-1,60-75
4,5,0,0,5.0,2.0,5.0,7.0,5.0,-1,-1,...,300,9100,44,-1,-1,1,-1,-1,-1,60-75


## Treinando MODELOS (4) para SEMANAS (18) para DURACAO

In [5]:
# inicializando listas para armazenar as métricas

for k in range (1,6): #iterando variacoes

    for y in range (1,5): #iteracao modelos

        #pulando a primeira semana "0" semana = x+1
        csemana = []
        cvariacao = []
        cvp = []
        cvn = []
        cfp = []
        cfn = []
        cacuracia = []
        cprecisao = []
        csensibilidade = []
        cf1escore = []
        for x in range(1,9): #iterando semanas
            semana = x

            if (k == 1): #2035
                df_test  = df_test2035.iloc[:, [3,4,5,6,7,13,14,15,16,17,23,24,25,26,27]].copy()
                df_trein = dfc_trein.iloc[0:semana*7].copy()
                df_trein = df_trein.iloc[:, [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
                dfc_test = df_test2035.copy()
                variacao = '20-35'

            if (k == 2): #40-55
                df_test  = df_test4055.iloc[:, [3,4,5,6,7,13,14,15,16,17,23,24,25,26,27]].copy()
                df_trein = dfc_trein.iloc[0:semana*7].copy()
                df_trein = df_trein.iloc[:, [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
                variacao = '40-55'

            if (k == 3): # 60-75
                df_test  = df_test6075.iloc[:, [3,4,5,6,7,13,14,15,16,17,23,24,25,26,27]].copy()
                df_trein = dfc_trein.iloc[0:semana*7].copy()
                df_trein = df_trein.iloc[:, [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
                variacao = '60-75'

            if (k == 4): #duracao 80-95
                df_test  = df_test8095.iloc[:, [3,4,5,6,7,13,14,15,16,17,23,24,25,26,27]].copy()
                df_trein = dfc_trein.iloc[0:semana*7].copy()
                df_trein = df_trein.iloc[:, [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
                variacao = '80-95'  

            if (k == 5): #duracao 20-95
                df_test  = df_test2095.iloc[:, [3,4,5,6,7,13,14,15,16,17,23,24,25,26,27]].copy()
                df_trein = dfc_trein.iloc[0:semana*7].copy()
                df_trein = df_trein.iloc[:, [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
                variacao = '20-95'                      


            #standarização
            scaler = StandardScaler().fit(df_trein)
            df_trein = scaler.transform(df_trein).copy()
            df_test = scaler.transform(df_test).copy()

            # aplicando o primeiro algoritmo o OSMV
            # fit the model
            if y == 1:
                clf = svm.OneClassSVM(nu=0.1, kernel='rbf', gamma='scale')
                algoritmo = 'ocsvm'
            if y == 2:
                clf = LocalOutlierFactor(n_neighbors=10, novelty=True)
                algoritmo = 'lof'
            if y == 3:
                clf = IsolationForest(contamination= 0.01, random_state=1)
                algoritmo = 'iforest'
            if y == 4:
                clf = EllipticEnvelope(contamination= 0.01, random_state=0, support_fraction= 0.9)
                algoritmo = 'rce'

            clf.fit(df_trein)
            y_pred_train = clf.predict(df_trein)
            y_pred_teste = clf.predict(df_test)
            n_error_train = y_pred_train[y_pred_train == -1].size

            # calculando métricas

            dfc_test['predicao'] = y_pred_teste
            dfc_test['vp'] = np.where((dfc_test['outlier'] == -1) & (dfc_test['predicao'] == -1), 1,0)
            dfc_test['vn'] = np.where((dfc_test['outlier'] == 1) &  (dfc_test['predicao'] == 1), 1,0)
            dfc_test['fp'] = np.where((dfc_test['outlier'] == -1) & (dfc_test['predicao'] == 1), 1,0)
            dfc_test['fn'] = np.where((dfc_test['outlier'] == 1) &  (dfc_test['predicao'] == -1), 1,0)
            VP = dfc_test['vp'].sum()
            VN = dfc_test['vn'].sum()
            FP = dfc_test['fp'].sum()
            FN = dfc_test['fn'].sum()


            erro = (FP+FN)/(VP+VN+FP+FN) #erro
            acuracia =  (VP+VN)/(VP+VN+FP+FN) # accuracy
            precisao = (VP)/(FP+VP) # precision
            sensibilidade = VP/(FN+VP) #recall
            f1escore = 2 * ((precisao * sensibilidade)/(precisao + sensibilidade))
            #armazenando metricas temp
            csemana.append(semana)
            cvariacao.append(variacao)
            cvp.append(VP)
            cvn.append(VN)
            cfp.append(FP)
            cfn.append(FN)
            cacuracia.append(acuracia)
            cprecisao.append(precisao)
            csensibilidade.append(sensibilidade)
            cf1escore.append(f1escore)
            # dentro do for X SEMANAS

        #criando um dataframe a partir das métricas    
        zipped = list(zip(csemana,cvariacao,cvp,cvn,cfp,cfn,cacuracia, cprecisao, csensibilidade, cf1escore))    
        dfmetricas = pd.DataFrame(zipped, columns=['semanas','variacao','vp', 'vn', 'fp', 'fn','acuracia','precisao','sensibilidade','f1score'])
        dfmetricas['algoritmo'] = algoritmo
        dfmetricas['variacao'] = variacao
        dfmetricas= dfmetricas.replace(np.nan,0)
        #print(dfmetricas.head(20))
        #exportando dataframes para datasets

        dfmetricas.to_csv(r'datasetaruba/results/armetricas-unifatrib-'+algoritmo+'-var-'+variacao+'-trein8.csv', sep=';', index=False, decimal=',')

        #dentro do for Y MODELOS
    #dentro do for Z ATRIBUTOS

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_lof.py:284: UserWarning: n_neighbors (10) is greater than the total number of samples (7). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_lof.py:284: UserWarning: n_neighbors (10) is greater than the total number of samples (7). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  wa

In [6]:
print('DONE')

DONE
